# Robot Framework Expert Fine-Tune (Unsloth)

This notebook fine-tunes a small instruct model using QLoRA and pushes the adapter to Hugging Face.

In [ ]:
# Dependencies are installed in the repo setup cell above.


In [ ]:
import os, subprocess, json
repo_url = "https://github.com/arvind3/robot-finetune-model"
repo_dir = "repo"
if not repo_url:
    raise SystemExit('REPO_URL is missing')
if os.path.isdir(repo_dir):
    !rm -rf {repo_dir}
!git clone {repo_url} {repo_dir}
%cd {repo_dir}
!git rev-parse --short HEAD
!pip -q install -r requirements-train.txt
print('Repo ready')


In [ ]:
import os
from getpass import getpass

token = os.environ.get('HF_TOKEN') or os.environ.get('HUGGINGFACE_TOKEN')
try:
    from google.colab import userdata
    token = token or userdata.get('HF_TOKEN')
except Exception:
    pass

if not token:
    token = getpass('Enter your Hugging Face token (HF_TOKEN): ')
os.environ['HF_TOKEN'] = token
print('HF_TOKEN set')


In [ ]:
from pathlib import Path
from datasets import load_dataset
dataset_repo = "arvind3/robotframework-expert-dataset"
train_path = Path('dataset/train.jsonl')
eval_path = Path('dataset/eval.jsonl')
if train_path.exists() and eval_path.exists():
    print('Using local dataset')
elif dataset_repo:
    print(f'Downloading dataset {dataset_repo}')
    ds = load_dataset(dataset_repo)
    Path('dataset').mkdir(exist_ok=True)
    ds['train'].to_json(train_path, orient='records', lines=True)
    ds['test' if 'test' in ds else 'validation' if 'validation' in ds else 'train'].to_json(eval_path, orient='records', lines=True)
else:
    print('Building dataset locally (validation-first)')
    import subprocess, sys
    proc = subprocess.run([sys.executable, 'tools/build_dataset.py'], check=False)
    if proc.returncode != 0:
        print('Validation build failed; retrying with --skip-validation')
        subprocess.run([sys.executable, 'tools/build_dataset.py', '--skip-validation'], check=True)
if not Path('data/eval_suite_v1.jsonl').exists():
    print('data/eval_suite_v1.jsonl missing; evaluator will use dataset/eval.jsonl fallback')

In [ ]:
base_model = "Qwen/Qwen2.5-3B-Instruct"
fallback_model = "Qwen/Qwen2.5-1.5B-Instruct"
hf_repo = "arvind3/robotframework-expert-qwen2.5-3b-lora"
seed = 42
!python train/train_unsloth.py --base-model {base_model} --fallback-model {fallback_model} --hf-repo {hf_repo} --seed {seed}
import pathlib, subprocess, sys
if not pathlib.Path('eval/comparison_metrics.json').exists():
    eval_runner = 'tools/run_eval_compat.py' if pathlib.Path('tools/run_eval_compat.py').exists() else 'tools/make_eval_report.py'
    cmd = [
        sys.executable, eval_runner,
        '--base-model', base_model,
        '--adapter-dir', 'outputs/adapter',
        '--merged-dir', 'outputs/merged',
        '--out-dir', 'eval',
        '--gate-mode', 'none',
        '--max-samples', '50',
    ]
    if pathlib.Path('data/eval_suite_v1.jsonl').exists():
        cmd.extend(['--eval-suite', 'data/eval_suite_v1.jsonl'])
    else:
        cmd.extend(['--dataset', 'dataset/eval.jsonl'])
    subprocess.run(cmd, check=True)
else:
    print('Comparative eval already generated by training script.')
if not pathlib.Path('outputs/run_artifacts.zip').exists():
    if pathlib.Path('tools/package_run_artifacts.py').exists():
        result = subprocess.run([
            sys.executable, 'tools/package_run_artifacts.py',
            '--eval-dir', 'eval',
            '--run-meta', 'outputs/run_meta.json',
            '--training-config', 'outputs/training_config.json',
            '--output-zip', 'outputs/run_artifacts.zip',
        ])
        if result.returncode != 0:
            print(f'[WARN] Artifact packaging failed (exit {result.returncode}); skipping.')
    else:
        print('[WARN] tools/package_run_artifacts.py not found; skipping artifact bundle.')
else:
    print('Artifact bundle already present.')
!if [ -f eval/report.md ]; then echo 'Comparative eval report: eval/report.md'; fi
!if [ -f outputs/run_artifacts.zip ]; then echo 'Artifact bundle: outputs/run_artifacts.zip'; fi